# Harmonization of MIDRC Imaging Study Descriptions using the LOINC Playbook
---
by Chris Meyer, PhD

Manager of Data and User Services at the Center for Translational Data Science at University of Chicago

November 2022
Last update December 2023

---
This Jupyter notebook demonstrates how to map the imaging_study node's `study_description` and `modality` properties to a `loinc_code` by utilizing the MIDRC DQH committee's [LOINC mapping table in GitHub](https://github.com/MIDRC/midrc_dicom_harmonization/tree/main/out). Once the imaging study has been mapped to a LOINC code, the code can be used to derive other LOINC properties. The following [LOINC properties](https://github.com/uc-cdis/midrc_dictionary/blob/deae581f0fb8b9ae5add1458d7882e189ba97af6/gdcdictionary/schemas/imaging_study.yaml#L77) are on the `imaging_study` node of the [MIDRC data dictionary](https://data.midrc.org/dd):
* loinc_code
* loinc_long_common_name
* loinc_method
* loinc_system
* loinc_contrast

## 1) Setup Python environment

In [1]:
# Import Python Packages and scripts
import pandas as pd
import numpy as np
from pathlib import Path
import sys, os, copy, datetime, shutil


import gen3
from gen3.submission import Gen3Submission
from gen3.auth import Gen3Auth
from gen3.index import Gen3Index
from gen3.query import Gen3Query


/Users/christopher/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# who's running the notebook
username = "christopher"

In [3]:
# download and import some custom Python scripts from https://github.com/cgmeyer/gen3sdk-python
loinc_dir = "/Users/{}/Documents/Notes/MIDRC/LOINC".format(username) #set your working dir
os.chdir(loinc_dir)
os.system("wget https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py -O expansion.py")
from expansion import Gen3Expansion


--2023-12-13 21:15:10--  https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 227011 (222K) [text/plain]
Saving to: ‘expansion.py’

     0K .......... .......... .......... .......... .......... 22% 1.48M 0s
    50K .......... .......... .......... .......... .......... 45% 3.50M 0s
   100K .......... .......... .......... .......... .......... 67% 2.31M 0s
   150K .......... .......... .......... .......... .......... 90% 12.1M 0s
   200K .......... .......... .                               100% 12.0M=0.07s

2023-12-13 21:15:10 (2.93 MB/s) - ‘expansion.py’ saved [227011/227011]



In [4]:
# You can view the SDK code/functions in GitHub: https://github.com/uc-cdis/gen3sdk-python
# Initiate instances of the Gen3 SDK Classes using credentials file downloaded from https://staging.midrc.org/identity
sapi = 'https://staging.midrc.org'
scred = '/Users/{}/Downloads/midrc-staging-credentials.json'.format(username)
sauth = Gen3Auth(sapi, refresh_file=scred) # authentication class
ssub = Gen3Submission(sapi, sauth) # submission class
squery = Gen3Query(sauth) # query class
sexp = Gen3Expansion(sapi,sauth,ssub) # class with some custom scripts
sexp.get_project_ids()
# Initiate instances of the Gen3 SDK Classes using credentials file downloaded from https://validatestaging.midrc.org/identity
vsapi = 'https://validatestaging.midrc.org'
vscred = '/Users/{}/Downloads/midrc-validatestaging-credentials.json'.format(username)
vsauth = Gen3Auth(vsapi, refresh_file=vscred) # authentication class
vssub = Gen3Submission(vsapi, vsauth) # submission class
vsquery = Gen3Query(vsauth) # query class
vsexp = Gen3Expansion(vsapi,vsauth,vssub) # class with some custom scripts
vsexp.get_project_ids()


Getting all project_ids you have access to in the data commons.
['Open-A1', 'Open-A1_PETAL_BLUECORAL', 'Open-A1_PETAL_REDCORAL', 'Open-A1_SCCM_VIRUS', 'Open-R1', 'TCIA-COVID-19-AR', 'TCIA-COVID-19-NY-SBU', 'TCIA-COVID-19_CT_Images', 'TCIA-RICORD']
Getting all project_ids you have access to in the data commons.
['SEQ_NCU-A4', 'SEQ_NCU-R4', 'SEQ_Open-A3', 'SEQ_Open-R3', 'SEQ_Restricted-DT']


['SEQ_NCU-A4', 'SEQ_NCU-R4', 'SEQ_Open-A3', 'SEQ_Open-R3', 'SEQ_Restricted-DT']

## 2) Prepare the mapping and filter tables
---
Get the mapping table and the filtering attributes tables from the [MIDRC GitHub repository](https://github.com/MIDRC/midrc_dicom_harmonization/tree/main/out) and reformat them to prepare for the LOINC mapping.

Prior to this, you will need to clone the repo using `git clone git@github.com:MIDRC/midrc_dicom_harmonization.git`


In [5]:
## Set the directory to your copy of the GitHub repo and pull main branch to pull latest updates from GitHub
git_dir = "/Users/{}/Documents/GitHub/MIDRC/midrc_dicom_harmonization/".format(username)
os.chdir(git_dir)
os.system("git checkout main")
os.system("git pull origin main")
os.chdir(loinc_dir)


Your branch is up to date with 'origin/main'.


Switched to branch 'main'


Updating 718a5ff..a19136f
Fast-forward
 in/StudyDescriptions_ACR.csv       |  764 ------------
 in/StudyDescriptions_Gen3.tsv      | 1758 ++++++++++++++++++++++++++
 in/StudyDescriptions_RSNA.csv      |  458 -------
 in/StudyDescriptions_TCIA.tsv      |   25 -
 pending/StudyDescription_diffs.csv | 2406 +++++++++++++++++++++---------------
 util/analyze_in_out.py             |   61 +-
 6 files changed, 3198 insertions(+), 2274 deletions(-)
 delete mode 100644 in/StudyDescriptions_ACR.csv
 create mode 100644 in/StudyDescriptions_Gen3.tsv
 delete mode 100644 in/StudyDescriptions_RSNA.csv
 delete mode 100644 in/StudyDescriptions_TCIA.tsv


From github.com:MIDRC/midrc_dicom_harmonization
 * branch            main       -> FETCH_HEAD
   718a5ff..a19136f  main       -> origin/main


In [6]:
## Create the output/working directory
now = datetime.datetime.now()
today = "{}-{}-{}".format(now.year, now.month, now.day)

results_dir = "{}/results_{}".format(loinc_dir,today)
Path(results_dir).mkdir(parents=True, exist_ok=True)

staging_results_dir = "{}/staging_results_{}".format(results_dir,today)
Path(staging_results_dir).mkdir(parents=True, exist_ok=True)

validatestaging_results_dir = "{}/validatestaging_results_{}".format(results_dir,today)
Path(validatestaging_results_dir).mkdir(parents=True, exist_ok=True)

In [7]:
mapping_file = "{}/out/StudyDescription_mapping_table.csv".format(git_dir)
mapping = pd.read_csv(mapping_file,dtype=str) #['Modality', 'StudyDescription', 'LOINC code', 'L-Long Common Name']

shutil.copy2(mapping_file, results_dir) # copy the mapping table version used for this mapping to the results_dir

mapping.rename(columns={"StudyDescription":"study_description","Modality":"study_modality","LOINC code":"loinc_code","L-Long Common Name":"loinc_long_common_name"},inplace=True)
mapping.drop_duplicates(inplace=True)
mapping['study_description'] = mapping.apply(lambda row: row['study_description'].casefold(),axis=1)
mapping['study_modality'] = mapping.apply(lambda row: row['study_modality'].casefold(),axis=1)

## Change any "(blank)" values for study_description to "[blank]" to match the mapping table
mapping.replace({"study_description":{
    '(blank)':'[blank]'}},
    inplace=True)

## remove any leading/trailing whitespaces in the codes/descriptions
mapping['loinc_code'] = mapping['loinc_code'].str.strip()
mapping['loinc_long_common_name'] = mapping['loinc_long_common_name'].str.strip()
mapping['study_description'] = mapping['study_description'].str.strip()
mapping['study_modality'] = mapping['study_modality'].str.strip()

## Summarize the mapping table
modalities = list(set(mapping['study_modality']))
descriptions = list(set(mapping['study_description']))
mapping_codes = list(set(mapping['loinc_code']))
mapping_names = list(set(mapping['loinc_long_common_name']))

display(mapping)
print("study_modality values in mapping table: {}".format(modalities))
print("Number of unique LOINC codes in mapping table: {}".format(len(mapping_codes)))
print("Number of unique LOINC Long Common Names in mapping table: {}".format(len(mapping_names)))


,study_modality,study_description,loinc_code,loinc_long_common_name
0,ct,[blank],25045-6,CT Unspecified body region
1,ct,pet ct fdg imag skull to thigh,81555-5,PET+CT Guidance for localization of tumor of W...
2,ct,chest pe(adult),79077-4,CTA Pulmonary arteries for pulmonary embolus W...
3,ct,ct chest pulmonary angio with iv con,79077-4,CTA Pulmonary arteries for pulmonary embolus W...
4,ct,ct chest pulmonary embolism (ctpe),79077-4,CTA Pulmonary arteries for pulmonary embolus W...
...,...,...,...,...
301,mg,[blank],36625-2,MG Breast Views
302,nm,[blank],49118-3,NM Unspecified body region Views
303,st,[blank],43526-3,SPECT Unspecified body region
304,[blank],bronchoscopy,18744-3,Bronchoscopy study


study_modality values in mapping table: ['cr, dx', 'rf', 'pt, ctpt', 'us', '[blank]', 'mr', 'ct', 'nm', 'st', 'mg']
Number of unique LOINC codes in mapping table: 83
Number of unique LOINC Long Common Names in mapping table: 83


In [8]:
filters_file = "{}/out/StudyDescription_filtering_attributes.csv".format(git_dir)
filters = pd.read_csv(filters_file,dtype=str) #['LOINC code', 'L-Long Common Name', 'L-Method', 'L-System', 'Rad.Timing']

shutil.copy2(filters_file, results_dir) # copy the mapping table version used for this mapping to the results_dir

filters.rename(columns= {
    'LOINC code':'loinc_code',
    'L-Long Common Name':'loinc_long_common_name',
    'L-Method':'loinc_method',
    'Rad.Timing':'loinc_contrast',
    'MIDRC-System':'loinc_system'
    },
    inplace=True,
    errors='ignore'
)
filters.drop(columns='L-System',inplace=True,errors='ignore')

## remove any leading/trailing whitespaces
filters['loinc_code'] = filters['loinc_code'].str.strip()
filters['loinc_long_common_name'] = filters['loinc_long_common_name'].str.strip()

filter_codes = list(set(filters['loinc_code']))
filter_names = list(set(filters['loinc_long_common_name']))
print("Number of unique LOINC codes in filters table: {}".format(len(filter_codes)))
print("Number of unique LOINC Long Common Names in mapping table: {}".format(len(filter_names)))


missing_codes = set(mapping_codes).difference(set(filter_codes))
missing_names = set(mapping_names).difference(set(filter_names))

print("Number of LOINC codes in mapping table missing from filters table: {}\n\t{}".format(len(missing_codes),missing_codes))
print("Number of LOINC names in mapping table missing from filters table: {}\n\t{}".format(len(missing_names),missing_names))

display(filters)


Number of unique LOINC codes in filters table: 82
Number of unique LOINC Long Common Names in mapping table: 82
Number of LOINC codes in mapping table missing from filters table: 1
	{'18744-3'}
Number of LOINC names in mapping table missing from filters table: 1
	{'Bronchoscopy study'}


,loinc_code,loinc_long_common_name,loinc_method,loinc_contrast,loinc_system
0,35889-5,RF Guidance for bronchoscopy of Chest,RF,NaN,Chest
1,36813-4,CT Abdomen and Pelvis W contrast IV,CT,W,Abdomen
2,42274-1,CT Abdomen and Pelvis WO and W contrast IV,CT,WO & W,Abdomen+Pelvis
3,36952-0,CT Abdomen and Pelvis WO contrast,CT,WO,Abdomen+Pelvis
4,79103-8,CT Abdomen W contrast IV,CT,W,Abdomen
...,...,...,...,...,...
77,83017-4,XR Chest View and Abdomen Supine and Upright,XR,NaN,Chest && Abdomen
78,30745-4,XR Chest Views,XR,NaN,Chest
79,24899-7,XR Ribs Views,XR,NaN,Chest>Ribs
80,43468-8,XR Unspecified body region Views,XR,NaN,Unspecified


## 3) Get the imaging_study information from MIDRC Staging and Validate Staging
---
* Use the [`Gen3Expansion.get_node_tsvs()` SDK function](https://github.com/cgmeyer/gen3sdk-python/blob/389e3945482439ace6e4536e6d0e35c6e48de9c9/expansion/expansion.py#L219) to get all the imaging_studies in MIDRC Staging (staging.midrc.org) and Validate Staging (validatestaging.midrc.org).

* The function `exp.get_node_tsvs()` will return a master dataframe containing all the imaging_study data from the projects specified. If no projects are specified, it will return all studies the user has access to.

In [9]:
os.chdir(loinc_dir)

# # The following limits the projects to those in program "Open"
# sprojects = sexp.get_project_ids(node='program',name=['Open'])
# sst = sexp.get_node_tsvs(node='imaging_study', overwrite=True, projects=sprojects,outdir=staging_results_dir)

# vsprojects = vsexp.get_project_ids(node='program',name=['SEQ_Open'])
# vst = vsexp.get_node_tsvs(node='imaging_study', overwrite=True, projects=vsprojects,outdir=validatestaging_results_dir)

sprojects = sexp.get_project_ids()
sst = sexp.get_node_tsvs(node='imaging_study', overwrite=True, projects=sprojects,outdir=staging_results_dir)

vsprojects = vsexp.get_project_ids()
vst = vsexp.get_node_tsvs(node='imaging_study', overwrite=True, projects=vsprojects,outdir=validatestaging_results_dir)


projects = sprojects + vsprojects



Getting all project_ids you have access to in the data commons.
['Open-A1', 'Open-A1_PETAL_BLUECORAL', 'Open-A1_PETAL_REDCORAL', 'Open-A1_SCCM_VIRUS', 'Open-R1', 'TCIA-COVID-19-AR', 'TCIA-COVID-19-NY-SBU', 'TCIA-COVID-19_CT_Images', 'TCIA-RICORD']

Output written to file: /Users/christopher/Documents/Notes/MIDRC/LOINC/results_2023-12-13/staging_results_2023-12-13/imaging_study_tsvs/Open-A1_imaging_study.tsv
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2023-12-13/staging_results_2023-12-13/imaging_study_tsvs/Open-A1_imaging_study.tsv has 57211 records.

Output written to file: /Users/christopher/Documents/Notes/MIDRC/LOINC/results_2023-12-13/staging_results_2023-12-13/imaging_study_tsvs/Open-A1_PETAL_BLUECORAL_imaging_study.tsv
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2023-12-13/staging_results_2023-12-13/imaging_study_tsvs/Open-A1_PETAL_BLUECORAL_imaging_study.tsv has 3496 records.

Output written to file: /Users/christopher/Documents/Notes/MIDRC/LOINC/results_2

In [10]:
# concat the open (staging) and sequestered (validatestaging) master imaging studies TSVs

st = pd.concat([sst,vst], ignore_index=False)
assert(len(st) == (len(sst)+len(vst)))

In [11]:
dupes = st.loc[st.duplicated(subset='submitter_id',keep=False)].sort_values(by='submitter_id')
print("There are {} duplicate submitter_ids in the list of imaging studies from Staging and ValidateStaging.".format(len(dupes)))
if len(dupes) > 0:
    display(dupes)


There are 102 duplicate submitter_ids in the list of imaging studies from Staging and ValidateStaging.


,type,id,project_id,submitter_id,age_at_imaging,age_at_imaging_gt89,body_part_examined,case_ids,days_from_study_to_neg_covid_test,days_from_study_to_pos_covid_test,...,loinc_method,loinc_system,study_description,study_location,study_modality,study_uid,study_year,study_year_shifted,cases.id,cases.submitter_id
106145,imaging_study,695252a4-d0dc-430a-b74a-0baa4248026a,Open-R1,1.2.826.0.1.3680043.10.474.419639.120025652450...,76.00,No,CHEST,419639-001882,"434,338,24,20,13","14,12,1,-16",...,XR,Chest,XR CHEST 1 VIEW AP,NaN,DX,1.2.826.0.1.3680043.10.474.419639.120025652450...,NaN,True,48345fc5-e90b-4486-ba82-c70343a87231,419639-001882
165125,imaging_study,8e67fffc-35fa-437a-84a0-4b56f9b56438,TCIA-RICORD,1.2.826.0.1.3680043.10.474.419639.120025652450...,75.76,No,CHEST,419639-001882,NaN,NaN,...,XR,Chest,XR CHEST 1 VIEW AP,NaN,DX,1.2.826.0.1.3680043.10.474.419639.120025652450...,2005.0,True,897df7c6-7046-444c-8de1-86c58c321e31,419639-001882
165532,imaging_study,e9b48dc2-4a70-4738-8791-cb2533014727,TCIA-RICORD,1.2.826.0.1.3680043.10.474.419639.125687583092...,69.48,No,CHEST,419639-000331,NaN,NaN,...,XR,Chest,XR CHEST 1 VIEW AP,NaN,DX,1.2.826.0.1.3680043.10.474.419639.125687583092...,2003.0,True,07f46765-3fcb-4f71-82cb-9cfab642f431,419639-000331
92033,imaging_study,3f24b2a2-0225-4613-bdb4-f58ce8f754b6,Open-R1,1.2.826.0.1.3680043.10.474.419639.125687583092...,69.00,No,CHEST,419639-000331,NaN,NaN,...,XR,Chest,XR CHEST 1 VIEW AP,NaN,DX,1.2.826.0.1.3680043.10.474.419639.125687583092...,NaN,True,30d8a16c-58e9-4991-8b4a-0f707e0b3867,419639-000331
165546,imaging_study,ec576dc5-2bac-40b5-ba6b-3739ff285a33,TCIA-RICORD,1.2.826.0.1.3680043.10.474.419639.141214871686...,50.61,No,CHEST,419639-000361,NaN,NaN,...,XR,Chest,XR CHEST 1 VIEW AP,NaN,DX,1.2.826.0.1.3680043.10.474.419639.141214871686...,2002.0,True,92a056ae-ad17-41b1-aa62-0fa78b252d84,419639-000361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122485,imaging_study,9aae6d7a-29fa-4685-9e0d-4d7a53083b9c,Open-R1,1.2.826.0.1.3680043.10.474.419639.797041384780...,69.00,No,CHEST,419639-000331,NaN,NaN,...,XR,Chest,XR CHEST 1 VIEW AP,NaN,DX,1.2.826.0.1.3680043.10.474.419639.797041384780...,NaN,True,30d8a16c-58e9-4991-8b4a-0f707e0b3867,419639-000331
87882,imaging_study,32cdb272-1c1d-4dae-a429-228f859c25a0,Open-R1,1.2.826.0.1.3680043.10.474.419639.876997626557...,69.00,No,CHEST,419639-000331,NaN,NaN,...,XR,Chest,XR CHEST 1 VIEW AP,NaN,DX,1.2.826.0.1.3680043.10.474.419639.876997626557...,NaN,True,30d8a16c-58e9-4991-8b4a-0f707e0b3867,419639-000331
164854,imaging_study,5401540b-7f76-4ad8-b2b8-3b613b082001,TCIA-RICORD,1.2.826.0.1.3680043.10.474.419639.876997626557...,69.48,No,CHEST,419639-000331,NaN,NaN,...,XR,Chest,XR CHEST 1 VIEW AP,NaN,DX,1.2.826.0.1.3680043.10.474.419639.876997626557...,2003.0,True,07f46765-3fcb-4f71-82cb-9cfab642f431,419639-000331
123744,imaging_study,9e88263d-a84e-4d70-b4fe-779d6b9bb2db,Open-R1,1.2.826.0.1.3680043.10.474.419639.981259836073...,51.00,NaN,CHEST,419639-003990,"84,22,20","6,-2,-3",...,XR,Chest,XR CHEST 1 VIEW AP,NaN,DX,1.2.826.0.1.3680043.10.474.419639.981259836073...,NaN,True,607b71c5-c7dd-4a26-a6d5-04cabc4999e4,419639-003990


### Create the "IN" file for DQH mapping team

As part of the MIDRC monthly release, the Gen3 team will create a pull request to deposit into the [IN folder of the MIDRC data harmonization GitHub repo](https://github.com/MIDRC/midrc_dicom_harmonization/tree/main/in) a file named "StudyDescriptions_Gen3.tsv" that has three columns: `Modality`, `StudyDescription`, and `frequency`

This TSV should contain the study_descriptions, modalities, and frequency count for *all studies* (both mapped and unmapped) across both the validatestaging and staging environments.

---

In [12]:
all_st = st[['study_modality','study_description']].groupby(["study_modality", "study_description"]).size().reset_index(name="frequency").sort_values(by='frequency',ascending=False)
all_st.rename(columns={'study_description':'StudyDescription','study_modality':'Modality'},inplace=True)
all_st

,Modality,StudyDescription,frequency
1540,DX,"XR CHEST 1 VW, FRONTAL",20016
1535,DX,XR CHEST 1 VIEW AP,19847
346,CR,XR PORT CHEST 1V,15011
1587,DX,XR CHEST PORTABLE 1 VIEW,12630
69,CR,CHEST PORT 1 VIEW (RAD)-CS,9164
...,...,...,...
870,CT,CT LOW-DOSE LUNG CANCER SCREEN,1
871,CT,CT LUMBAR SPINE WO IV CONTRAST,1
876,CT,CT ORBITS OPTIC NERVES WO CONTRAST,1
877,CT,CT ORBITS OPTIC NERVES WWO CONTRAST,1


In [13]:
# "StudyDescriptions_Gen3.tsv" that has three columns: `Modality`, `StudyDescription`, and `frequency`
filename="{}/StudyDescriptions_Gen3.tsv".format(results_dir)
all_st.to_csv(filename, sep='\t', index=False)
print("Frequencies of all imaging study description/modality combinations for projects {} saved to file: \n{}\n".format(projects,filename))



Frequencies of all imaging study description/modality combinations for projects ['Open-A1', 'Open-A1_PETAL_BLUECORAL', 'Open-A1_PETAL_REDCORAL', 'Open-A1_SCCM_VIRUS', 'Open-R1', 'TCIA-COVID-19-AR', 'TCIA-COVID-19-NY-SBU', 'TCIA-COVID-19_CT_Images', 'TCIA-RICORD', 'SEQ_NCU-A4', 'SEQ_NCU-R4', 'SEQ_Open-A3', 'SEQ_Open-R3', 'SEQ_Restricted-DT'] saved to file: 
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2023-12-13/StudyDescriptions_Gen3.tsv



## 4) Reformat the imaging_study data to prepare for mapping
---
* **NaN values**: In the mapping table, a `study_description` or `study_modality` with a value of `null` or `NaN` is listed as `[blank]`.
* **Case-insensitivity**: we should ignore capitalization of `study_modality` and `study_description` strings when we do the look-up. We'll use `str.casefold()` to ignore case for matching.
* **CR/DX Modality**: in the mapping table, all versions of "CR" and "DX" modalities are denoted "CR, DX"; so, we need to change all occurrences of other spellings to match that exactly.
* **Duplicated study UIDs**: Identify any duplicated study UIDs and remove them from studies.


In [14]:
## Make a copy of the master imaging_study table with only data relevant to LOINC mapping, and change NaN to "[blank]"
studies = copy.deepcopy(st[['submitter_id','study_description','study_modality']])
#studies.index = studies['submitter_id']
studies['study_description'].fillna("[blank]", inplace=True)
studies['study_modality'].fillna("[blank]", inplace=True)

## Make the mapping case-insensitive
studies['study_description'] = studies['study_description'].str.casefold()
studies['study_modality'] = studies['study_modality'].str.casefold()

## remove any leading/trailing whitespaces
studies['study_description'] = studies['study_description'].str.strip()
studies['study_modality'] = studies['study_modality'].str.strip()


## Fix any CR/DX study_modality to all be "cr, dx" to match mapping table
studies.replace({"study_modality":{
    'cr':'cr, dx',
    'dx':'cr, dx',
    'cr,dx':'cr, dx',
    'dx,cr':'cr, dx',
    'dx, cr':'cr, dx',
    'crdx':'cr, dx',
    'dxcr':'cr, dx'}},
    inplace=True)

## Fix any CT/PT study_modality to all be "pt, ct" to match mapping table
studies.replace({"study_modality":{
    'pt':'pt, ctpt',
    'ctpt':'pt, ctpt',
    'ptct':'pt, ctpt',
    'ct, pt':'pt, ctpt',
    'pt, ct':'pt, ctpt',
    'pt,ct':'pt, ctpt',
    'ct,pt':'pt, ctpt'}},
    inplace=True)

## Change any "<NONE>" values for study_description to "[blank]" to match the mapping table
studies.replace({"study_description":{
    '<none>':'[blank]'}},
    inplace=True)

## Identify and drop duplicates
dupes = studies.loc[studies.duplicated(keep=False)]
studies = studies.drop_duplicates()
display(studies)



,submitter_id,study_description,study_modality
0,2.16.840.1.114274.1818.52299511531607822051447...,xr chest pa/lateral,"cr, dx"
1,2.16.840.1.114274.1818.51555196924311533821064...,chest port 1 view (rad)-cs,"cr, dx"
2,2.16.840.1.114274.1818.53927660811344067041430...,xr chest pa and lateral,"cr, dx"
3,2.16.840.1.114274.1818.46320552695846431151951...,xr chest 2 views,"cr, dx"
4,2.16.840.1.114274.1818.56139956906507085344525...,xr port chest 1v,"cr, dx"
...,...,...,...
37378,1.2.826.0.1.3680043.10.474.671385.27856,[blank],"cr, dx"
37379,1.2.826.0.1.3680043.10.474.514382.5247412,"xr chest 1 vw, frontal","cr, dx"
37380,1.2.826.0.1.3680043.10.474.419639.468322285297...,xr chest 1 view ap,"cr, dx"
37381,1.2.826.0.1.3680043.10.474.514382.1329285,"xr chest 1 vw, frontal","cr, dx"


In [15]:
## Check modalities for presence in the mapping table and frequency in MIDRC data:
study_modalities = list(set(studies.study_modality))
print("There are the following values of study_modality in the MIDRC imaging_study data: \n{}".format(study_modalities))

missing_modalities = set(study_modalities).difference(modalities)

mfreq = {}
for mod in missing_modalities:
    freq = len(studies.loc[studies['study_modality']==mod])
    mfreq[mod] = freq

print("\nThese modalities are in the study data but not in the mapping table: \n{}".format(mfreq))

There are the following values of study_modality in the MIDRC imaging_study data: 
['cr, dx', 'sr', 'rf', 'ot,pt', 'pt, ctpt', 'us', 'dr', 'pr', 'mr', 'ct', 'xa', 'nm', 'mg']

These modalities are in the study data but not in the mapping table: 
{'sr': 2, 'ot,pt': 2, 'pr': 1, 'dr': 2, 'xa': 1}


## 5) Do mapping using pandas pd.merge(): 
---

1) Merge `loinc_code` in `mapping` dataframe into the `studies` dataframe on the combination of `study_description` and `study_modality`.

2) Merge the `loinc_long_common_name`, `loinc_method`, `loinc_contrast`, and `loinc_system` in `filters` dataframe into the `studies` dataframe on `loinc_code`.



In [16]:
sdf = studies.merge(mapping,on=['study_modality','study_description'],how='left').drop_duplicates()
sdf = sdf.merge(filters,on=['loinc_code','loinc_long_common_name'],how='left')
sdf.drop(columns=['study_description','study_modality'],inplace=True,errors='ignore')
sdf

,submitter_id,loinc_code,loinc_long_common_name,loinc_method,loinc_contrast,loinc_system
0,2.16.840.1.114274.1818.52299511531607822051447...,42272-5,XR Chest PA and Lateral,XR,NaN,Chest
1,2.16.840.1.114274.1818.51555196924311533821064...,36589-0,Portable XR Chest AP single view,XR.portable,NaN,Chest
2,2.16.840.1.114274.1818.53927660811344067041430...,36643-5,XR Chest 2 Views,XR,NaN,Chest
3,2.16.840.1.114274.1818.46320552695846431151951...,42272-5,XR Chest PA and Lateral,XR,NaN,Chest
4,2.16.840.1.114274.1818.56139956906507085344525...,36589-0,Portable XR Chest AP single view,XR.portable,NaN,Chest
...,...,...,...,...,...,...
202968,1.2.826.0.1.3680043.10.474.671385.27856,43468-8,XR Unspecified body region Views,XR,NaN,Unspecified
202969,1.2.826.0.1.3680043.10.474.514382.5247412,36554-4,XR Chest Single view,XR,NaN,Chest
202970,1.2.826.0.1.3680043.10.474.419639.468322285297...,36572-6,XR Chest AP,XR,NaN,Chest
202971,1.2.826.0.1.3680043.10.474.514382.1329285,36554-4,XR Chest Single view,XR,NaN,Chest


In [17]:
## Pull in the original study_modality and study_description (non-lowercase) and rearrange the columns
cols = ['type','project_id','submitter_id','cases.submitter_id','study_modality','study_description','loinc_code','loinc_long_common_name','loinc_method','loinc_contrast','loinc_system']
sdf = sdf.merge(st[['type','project_id','submitter_id','cases.submitter_id','study_description','study_modality']], on='submitter_id',how='left')[cols]
sdf


,type,project_id,submitter_id,cases.submitter_id,study_modality,study_description,loinc_code,loinc_long_common_name,loinc_method,loinc_contrast,loinc_system
0,imaging_study,Open-A1,2.16.840.1.114274.1818.52299511531607822051447...,10003752-sh69V58TQkafSp4b8xTdw,CR,XR CHEST PA/LATERAL,42272-5,XR Chest PA and Lateral,XR,NaN,Chest
1,imaging_study,Open-A1,2.16.840.1.114274.1818.51555196924311533821064...,10008204-ASGg9enGJ0K2IEZmipuabQ,CR,CHEST PORT 1 VIEW (RAD)-CS,36589-0,Portable XR Chest AP single view,XR.portable,NaN,Chest
2,imaging_study,Open-A1,2.16.840.1.114274.1818.53927660811344067041430...,10000364-1375261,DX,XR CHEST PA AND LATERAL,36643-5,XR Chest 2 Views,XR,NaN,Chest
3,imaging_study,Open-A1,2.16.840.1.114274.1818.46320552695846431151951...,10041569-KRZhwghggkKf11HcSXpS5A,DX,XR CHEST 2 VIEWS,42272-5,XR Chest PA and Lateral,XR,NaN,Chest
4,imaging_study,Open-A1,2.16.840.1.114274.1818.56139956906507085344525...,10003752-tXQfrz4Lxkdk8w8sHZxVA,CR,XR PORT CHEST 1V,36589-0,Portable XR Chest AP single view,XR.portable,NaN,Chest
...,...,...,...,...,...,...,...,...,...,...,...
203019,imaging_study,SEQ_Open-R3,1.2.826.0.1.3680043.10.474.671385.27856,639127-003297,DX,NaN,43468-8,XR Unspecified body region Views,XR,NaN,Unspecified
203020,imaging_study,SEQ_Open-R3,1.2.826.0.1.3680043.10.474.514382.5247412,514382-035056,CR,"XR CHEST 1 VW, FRONTAL",36554-4,XR Chest Single view,XR,NaN,Chest
203021,imaging_study,SEQ_Open-R3,1.2.826.0.1.3680043.10.474.419639.468322285297...,419639-010529,DX,XR CHEST 1 VIEW AP,36572-6,XR Chest AP,XR,NaN,Chest
203022,imaging_study,SEQ_Open-R3,1.2.826.0.1.3680043.10.474.514382.1329285,514382-012543,DX,"XR CHEST 1 VW, FRONTAL",36554-4,XR Chest Single view,XR,NaN,Chest


In [18]:
mdf = sdf.loc[~sdf['loinc_code'].isna()]
udf = sdf.loc[sdf['loinc_code'].isna()]
fdf = udf.groupby(["study_modality", "study_description"]).size().reset_index(name="Freq").sort_values(by='Freq',ascending=False)
print("Total Studies: {}, Mapped: {}, Unmapped: {}".format(len(sdf),len(mdf),len(udf)))

Total Studies: 203024, Mapped: 189133, Unmapped: 13891


In [19]:
## Save results to files
filename="{}/LOINC_mapping_results_{}_{}.tsv".format(results_dir,len(sdf),today)
sdf.to_csv(filename, sep='\t', index=False)
print("All LOINC mapping results for projects {} saved to file: \n{}\n".format(projects,filename))

filename="{}/LOINC_mapped_{}_{}.tsv".format(results_dir,len(mdf),today)
mdf.to_csv(filename, sep='\t', index=False)
print("Successfully mapped imaging studies for projects {} saved to file: \n{}\n".format(projects,filename))

filename="{}/LOINC_umapped_{}_{}.tsv".format(results_dir,len(udf),today)
udf.to_csv(filename, sep='\t', index=False)
print("Unmapped imaging studies for projects {} saved to file: \n{}\n".format(projects,filename))

filename="{}/LOINC_umapped_frequencies_{}_{}.tsv".format(results_dir,len(fdf),today)
fdf.to_csv(filename, sep='\t', index=False)
print("Frequencies of unmapped imaging study description/modality combinations for projects {} saved to file: \n{}\n".format(projects,filename))


All LOINC mapping results for projects ['Open-A1', 'Open-A1_PETAL_BLUECORAL', 'Open-A1_PETAL_REDCORAL', 'Open-A1_SCCM_VIRUS', 'Open-R1', 'TCIA-COVID-19-AR', 'TCIA-COVID-19-NY-SBU', 'TCIA-COVID-19_CT_Images', 'TCIA-RICORD', 'SEQ_NCU-A4', 'SEQ_NCU-R4', 'SEQ_Open-A3', 'SEQ_Open-R3', 'SEQ_Restricted-DT'] saved to file: 
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2023-12-13/LOINC_mapping_results_203024_2023-12-13.tsv

Successfully mapped imaging studies for projects ['Open-A1', 'Open-A1_PETAL_BLUECORAL', 'Open-A1_PETAL_REDCORAL', 'Open-A1_SCCM_VIRUS', 'Open-R1', 'TCIA-COVID-19-AR', 'TCIA-COVID-19-NY-SBU', 'TCIA-COVID-19_CT_Images', 'TCIA-RICORD', 'SEQ_NCU-A4', 'SEQ_NCU-R4', 'SEQ_Open-A3', 'SEQ_Open-R3', 'SEQ_Restricted-DT'] saved to file: 
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2023-12-13/LOINC_mapped_189133_2023-12-13.tsv

Unmapped imaging studies for projects ['Open-A1', 'Open-A1_PETAL_BLUECORAL', 'Open-A1_PETAL_REDCORAL', 'Open-A1_SCCM_VIRUS', 'Open-R1', 'TCIA

In [20]:
## Take a glance at frequency of the unmapped imaging study modality/description combinations
display(fdf)

,study_modality,study_description,Freq
374,CT,BRAIN W/O CONTRAST (CT)-CS,2029
1141,DX,DX CHEST PORTABLE 1 VIEW,1431
75,CR,DX CHEST 1 VIEW,1046
84,CR,DX CHEST PORTABLE 1 VIEW,890
1256,DX,XR Chest 2 Views PA + Lat Stnd Protocol,796
...,...,...,...
633,CT,CT CHEST/ABDOMEN/PELVIS W/CONTRAST,1
634,CT,CT CHST/ABD/PEL W/IV CONTRAST,1
635,CT,CT CODE STROKE CTA,1
638,CT,CT CT-Abdomen/Pelvis W/Contrast,1


## 6) Submit the data to Staging and ValidateStaging
---
Use the Gen3SDK function `Gen3Submission.submit_file()` to update the LOINC properties for each of the newly mapped studies in `mdf`.

In [21]:
# let's first seperate the mapped studies in mdf into open and seq

# open
smdf = mdf.loc[mdf.project_id.isin(sprojects)].reset_index(drop=True)

# sequestered
vsmdf = mdf.loc[mdf.project_id.isin(vsprojects)].reset_index(drop=True)

In [ ]:
# Submit the derived data to validatestaging

projects = list(set(vsmdf['project_id']))
vsdata = {}
for vspid in projects:
    print("Submitting data to project '{}'.".format(vspid))
    vsdata[vspid] = vsexp.submit_df(df=vsmdf.loc[vsmdf["project_id"]==vspid],project_id=vspid, chunk_size=1000)

In [22]:
# Submit the derived data to staging

projects = list(set(smdf['project_id']))
sdata = {}
for spid in projects:
    print("Submitting data to project '{}'.".format(spid))
    sdata[spid] = sexp.submit_df(df=smdf.loc[smdf["project_id"]==spid],project_id=spid, chunk_size=200)

Submitting data to project 'Open-A1_PETAL_REDCORAL'.
Submitting ['imaging_study'] DataFrame with 3478 records.
	Chunk 1 (chunk size: 200, submitted: 0 of 3478)


/Users/christopher/Documents/Notes/MIDRC/LOINC/expansion.py:1981: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(


	 Succeeded: 200 entities.
	Chunk 2 (chunk size: 200, submitted: 200 of 3478)
	 Succeeded: 200 entities.
	Chunk 3 (chunk size: 200, submitted: 400 of 3478)
	 Succeeded: 200 entities.
	Chunk 4 (chunk size: 200, submitted: 600 of 3478)
	 Succeeded: 200 entities.
	Chunk 5 (chunk size: 200, submitted: 800 of 3478)
	 Succeeded: 200 entities.
	Chunk 6 (chunk size: 200, submitted: 1000 of 3478)
	 Succeeded: 200 entities.
	Chunk 7 (chunk size: 200, submitted: 1200 of 3478)
	 Succeeded: 200 entities.
	Chunk 8 (chunk size: 200, submitted: 1400 of 3478)
	 Succeeded: 200 entities.
	Chunk 9 (chunk size: 200, submitted: 1600 of 3478)
	 Succeeded: 200 entities.
	Chunk 10 (chunk size: 200, submitted: 1800 of 3478)
	 Succeeded: 200 entities.
	Chunk 11 (chunk size: 200, submitted: 2000 of 3478)
	 Succeeded: 200 entities.
	Chunk 12 (chunk size: 200, submitted: 2200 of 3478)
	 Succeeded: 200 entities.
	Chunk 13 (chunk size: 200, submitted: 2400 of 3478)
	 Succeeded: 200 entities.
	Chunk 14 (chunk size: 20

## The End
If you have any questions reach out to the Gen3 MIDRC team or cgmeyer@uchicago.edu


{'Open-A1_PETAL_REDCORAL': {'invalid': {},
  'other': [],
  'details': [],
  'succeeded': ['2.16.840.1.114274.1818.5676201508537179876189239948566900361',
   '2.16.840.1.114274.1818.56529960447387442836307556660838220471',
   '2.16.840.1.114274.1818.4809532397463610267723261835794633616',
   '2.16.840.1.114274.1818.52885750876170542498925481011477906870',
   '2.16.840.1.114274.1818.554043509389745182512988286000149764024',
   '2.16.840.1.114274.1818.57003217332340952447220059088224413087',
   '2.16.840.1.114274.1818.504141045643739244111453646398843749800',
   '2.16.840.1.114274.1818.552940431717437888412488087698467712936',
   '2.16.840.1.114274.1818.491982442268067718017301744110081325707',
   '2.16.840.1.114274.1818.55072178592247456124962635812184842369',
   '2.16.840.1.114274.1818.5758551462205078074436129142480264337',
   '2.16.840.1.114274.1818.548778781655049964414348473606256392872',
   '2.16.840.1.114274.1818.483227892574065146512616324804490966410',
   '2.16.840.1.114274.181